In [16]:
# Dependencies
import pandas as pd
import requests
import json
import os
import datetime
import matplotlib.pyplot as plt

# Google API Key (maybe later)
from config import gkey


In [4]:
# Source files
attendance_raw = 'Resources/national_park_attendance_Prev3Y.xlsx'
park_info = 'Resources/national_park_locations.xlsx'
#wildfires = 'Resources/California_Fire_Incidents.csv'

# Read data and store in dataframe
attendance_data = pd.read_excel(attendance_raw)
park_info_data = pd.read_excel(park_info)
#wildfire_data = pd.read_csv(wildfires)


In [6]:
# Limit dataframe to California & Florifa
national_parks = park_info_data.loc[(park_info_data['ParkType']=='National Park')]
focus_parks = pd.DataFrame(national_parks.loc[(park_info_data['State'] == 'CA') | (park_info_data['State'] == 'FL')])
focus_parks.reset_index()

#focus_parks['FullName'] = focus_parks['Park']

# Create new 'FullName' column (empty)
focus_parks['FullName'] = focus_parks['Park'].str.replace(['NP'],'')#+focus_parks['ParkType'])

focus_parks['FullName']=focus_parks['Park'].str[:-2]+focus_parks['ParkType']

#focus_parks['Park'].replace('NP', 'National Park', inplace=True)
#focus_parks['Park'] = focus_parks['Park'].str.replace('NP','')

focus_parks.head(20)

,Park,UnitCode,ParkType,Region,State,FullName
30,Biscayne NP,BISC,National Park,Southeast,FL,Biscayne National Park
63,Channel Islands NP,CHIS,National Park,Pacific West,CA,Channel Islands National Park
87,Death Valley NP,DEVA,National Park,Pacific West,CA,Death Valley National Park
93,Dry Tortugas NP,DRTO,National Park,Southeast,FL,Dry Tortugas National Park
101,Everglades NP,EVER,National Park,Southeast,FL,Everglades National Park
191,Joshua Tree NP,JOTR,National Park,Pacific West,CA,Joshua Tree National Park
198,Kings Canyon NP,KICA,National Park,Pacific West,CA,Kings Canyon National Park
210,Lassen Volcanic NP,LAVO,National Park,Pacific West,CA,Lassen Volcanic National Park
281,Pinnacles NP,PINN,National Park,Pacific West,CA,Pinnacles National Park
293,Redwood NP,REDW,National Park,Pacific West,CA,Redwood National Park


In [7]:
# Find the geocoordinates (latitude and longitude)
park = "Yellowstone"

params = {"address": park, "key": gkey}

# Build URL using the Google Maps API
base_url = "https://maps.googleapis.com/maps/api/geocode/json"

#print("Drill #1: The Geocoordinates of X")

# Run request
response = requests.get(base_url, params=params)

# print the response URL, avoid doing for public GitHub repos in order to avoid exposing key
# print(response.url)

# Convert to JSON
park_geo = response.json()

# Extract lat/lng
lat = park_geo["results"][0]["geometry"]["location"]["lat"]
lng = park_geo["results"][0]["geometry"]["location"]["lng"]

# Print results
print(f"{park}: {lat}, {lng}")

Yellowstone: 44.427963, -110.588455


In [8]:
# Build URL using the Google Maps API

lats=[]
longs=[]
cost=[]

base_url = "https://maps.googleapis.com/maps/api/geocode/json"

for Park in focus_parks['FullName']:
    params = {"address": Park, "key": gkey}
    response = requests.get(base_url, params=params)
    park_geo = response.json()
    lats.append(park_geo["results"][0]["geometry"]["location"]["lat"])
    longs.append(park_geo["results"][0]["geometry"]["location"]["lng"])           
    

In [9]:
# Add geocoordinates to dataframe
focus_parks['Latitude']=lats
focus_parks['Longitude']=longs
focus_parks.head()

,Park,UnitCode,ParkType,Region,State,FullName,Latitude,Longitude
30,Biscayne NP,BISC,National Park,Southeast,FL,Biscayne National Park,25.482423,-80.208310
63,Channel Islands NP,CHIS,National Park,Pacific West,CA,Channel Islands National Park,34.006936,-119.778533
87,Death Valley NP,DEVA,National Park,Pacific West,CA,Death Valley National Park,36.505389,-117.079408
93,Dry Tortugas NP,DRTO,National Park,Southeast,FL,Dry Tortugas National Park,24.633333,-82.920000
101,Everglades NP,EVER,National Park,Southeast,FL,Everglades National Park,25.745929,-80.554956


In [14]:
# Combine the data into a single dataframe  
park_data = pd.merge(focus_parks, attendance_data, how="left", on=["Park", "Park"])
park_data.head()

,Park,UnitCode,ParkType,Region,State,FullName,Latitude,Longitude,Code,Month,Visitors
0,Biscayne NP,BISC,National Park,Southeast,FL,Biscayne National Park,25.482423,-80.20831,BISC,2020-01-01 00:00:00,79041.0
1,Biscayne NP,BISC,National Park,Southeast,FL,Biscayne National Park,25.482423,-80.20831,BISC,2020-02-01 00:00:00,91655.0
2,Biscayne NP,BISC,National Park,Southeast,FL,Biscayne National Park,25.482423,-80.20831,BISC,2020-03-01 00:00:00,10962.0
3,Biscayne NP,BISC,National Park,Southeast,FL,Biscayne National Park,25.482423,-80.20831,BISC,2020-04-01 00:00:00,2358.0
4,Biscayne NP,BISC,National Park,Southeast,FL,Biscayne National Park,25.482423,-80.20831,BISC,2020-05-01 00:00:00,43692.0


In [29]:
park_data['Calendar Year'] = pd.DatetimeIndex(park_data['Month']).year
park_data['Calendar Month'] = pd.DatetimeIndex(park_data['Month']).month
park_data.head(50)

,Park,UnitCode,ParkType,Region,State,FullName,Latitude,Longitude,Code,Month,Visitors,Year,Calandar Year,Calendar Month,Calendar Year
0,Biscayne NP,BISC,National Park,Southeast,FL,Biscayne National Park,25.482423,-80.208310,BISC,2020-01-01 00:00:00,79041.0,2020,2020,1,2020
1,Biscayne NP,BISC,National Park,Southeast,FL,Biscayne National Park,25.482423,-80.208310,BISC,2020-02-01 00:00:00,91655.0,2020,2020,2,2020
2,Biscayne NP,BISC,National Park,Southeast,FL,Biscayne National Park,25.482423,-80.208310,BISC,2020-03-01 00:00:00,10962.0,2020,2020,3,2020
3,Biscayne NP,BISC,National Park,Southeast,FL,Biscayne National Park,25.482423,-80.208310,BISC,2020-04-01 00:00:00,2358.0,2020,2020,4,2020
4,Biscayne NP,BISC,National Park,Southeast,FL,Biscayne National Park,25.482423,-80.208310,BISC,2020-05-01 00:00:00,43692.0,2020,2020,5,2020
5,Biscayne NP,BISC,National Park,Southeast,FL,Biscayne National Park,25.482423,-80.208310,BISC,2020-06-01 00:00:00,32646.0,2020,2020,6,2020
6,Biscayne NP,BISC,National Park,Southeast,FL,Biscayne National Park,25.482423,-80.208310,BISC,2020-07-01 00:00:00,28770.0,2020,2020,7,2020
7,Biscayne NP,BISC,National Park,Southeast,FL,Biscayne National Park,25.482423,-80.208310,BISC,2020-08-01 00:00:00,34416.0,2020,2020,8,2020
8,Biscayne NP,BISC,National Park,Southeast,FL,Biscayne National Park,25.482423,-80.208310,BISC,2020-09-01 00:00:00,18276.0,2020,2020,9,2020
9,Biscayne NP,BISC,National Park,Southeast,FL,Biscayne National Park,25.482423,-80.208310,BISC,2020-10-01 00:00:00,17436.0,2020,2020,10,2020


In [39]:
# Split up our data into groups based upon 'bikeid' and 'gender'
year_groups = park_data.groupby(['Calendar Year','Calendar Month'])

# Create a new variable that holds the sum of our groups
sum_it_up = year_groups.sum()
sum_it_up.head(50)

Latitude    Longitude   Visitors   Year  \
Calendar Year Calendar Month                                              
2017          1               409.688152 -1319.709203   766308.0  24204   
              2               409.688152 -1319.709203   713317.0  24204   
              3               409.688152 -1319.709203  1016798.0  24204   
              4               409.688152 -1319.709203  1161838.0  24204   
              5               409.688152 -1319.709203  1284854.0  24204   
              6               409.688152 -1319.709203  1353126.0  24204   
              7               409.688152 -1319.709203  1652030.0  24204   
              8               409.688152 -1319.709203  1583956.0  24204   
              9               409.688152 -1319.709203  1296326.0  24204   
              10              409.688152 -1319.709203  1082981.0  24204   
              11              409.688152 -1319.709203   865442.0  24204   
              12              409.688152 -1319.709203   781624.0  24204   
2018          1               409.688152 -1319.709203   698100.0  24216   
              2               409.688152 -1319.709203   791302.0  24216   
              3               409.688152 -1319.709203  1009598.0  24216   
              4               409.688152 -1319.709203  1078583.0  24216   
              5               409.688152 -1319.709203  1200857.0  24216   
              6               409.688152 -1319.709203  1358084.0  24216   
              7               409.688152 -1319.709203  1441211.0  24216   
              8               409.688152 -1319.709203  1349803.0  24216   
              9               409.688152 -1319.709203  1297494.0  24216   
              10              409.688152 -1319.709203  1105785.0  24216   
              11              409.688152 -1319.709203   904695.0  24216   
              12              409.688152 -1319.709203  1017143.0  24216   
2019          1               409.688152 -1319.709203   660024.0  24228   
              2               409.688152 -1319.709203   745755.0  24228   
              3               409.688152 -1319.709203  1236231.0  24228   
              4               409.688152 -1319.709203  1288180.0  24228   
              5               409.688152 -1319.709203  1288243.0  24228   
              6               409.688152 -1319.709203  1396080.0  24228   
              7               409.688152 -1319.709203  1662196.0  24228   
              8               409.688152 -1319.709203  1645794.0  24228   
              9               409.688152 -1319.709203  1435499.0  24228   
              10              409.688152 -1319.709203  1288878.0  24228   
              11              409.688152 -1319.709203   985016.0  24228   
              12              409.688152 -1319.709203   913257.0  24228   
2020          1               409.688152 -1319.709203   822438.0  24240   
              2               409.688152 -1319.709203   996423.0  24240   
              3               409.688152 -1319.709203   514349.0  24240   
              4               409.688152 -1319.709203    32645.0  24240   
              5               409.688152 -1319.709203   209919.0  24240   
              6               409.688152 -1319.709203   797805.0  24240   
              7               409.688152 -1319.709203  1254779.0  24240   
              8               409.688152 -1319.709203  1089101.0  24240   
              9               409.688152 -1319.709203   515273.0  24240   
              10              409.688152 -1319.709203   779808.0  24240   
              11              409.688152 -1319.709203   995919.0  24240   

                              Calandar Year  
Calendar Year Calendar Month                 
2017          1                       24204  
              2                       24204  
              3                       24204  
              4                       24204  
              5                       24204  
              6                   

In [27]:
# Generate a summary statistics table of mean, median, variance, standard deviation, and SEM of the visutors for each calendar year
# Use groupby and summary statistical methods to calculate the following properties of each drug regimen: 
# mean, median, variance, standard deviation, and SEM of the tumor volume. 


mean = park_data.groupby('Calendar Month').mean()['Visitors']
median = park_data.groupby('Calendar Month').median()['Visitors']
variance = park_data.groupby('Calendar Month').var()['Visitors']
std_dev = park_data.groupby('Calendar Month').std()['Visitors']
SEM = park_data.groupby('Calendar Month').sem()['Visitors']

# Assemble the resulting series into a single summary dataframe.
park_stats = pd.DataFrame({'Mean': mean,
              'Median':median,
              'Variance': variance,
              'Standard Deviation': std_dev,
                         'SEM':SEM})
park_stats.head(12)

,Mean,Median,Variance,Standard Deviation,SEM
Calendar Month,,,,,
1,61393.125000,26234.0,5.287437e+09,72714.764692,10495.472242
2,67641.604167,26092.0,6.314422e+09,79463.338589,11469.544981
3,78687.000000,30803.0,1.175751e+10,108432.049317,15650.818215
4,74192.625000,34330.5,1.086069e+10,104214.648238,15042.088803
5,82997.354167,41381.0,1.157948e+10,107608.006993,15531.877951
6,102189.479167,66044.5,1.572199e+10,125387.345128,18098.104366
7,125212.833333,71268.5,2.400965e+10,154950.472300,22365.174223
8,118096.958333,82618.0,2.083480e+10,144342.638131,20834.065245
9,94679.000000,49345.5,1.742104e+10,131988.775982,19050.938836


In [ ]:
# Generate a summary statistics table of mean, median, variance, standard deviation, and SEM of the tumor volume for each regimen
# Using the aggregation method, produce the same summary statistics in a single line
# Google take the wheel ...
summary_stats= park_data.groupby('Park').agg({'Visitors':['mean','median','var','std','sem']})
summary_stats